### Реализация стекинга с использованием класса

In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [3]:
from xgboost import XGBClassifier, XGBRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_predict

from sklearn.datasets import make_classification

In [4]:
from easy_stacking import EasyStacking

In [5]:
# Код для автообновления функций (запустить один раз)
%load_ext autoreload
%autoreload 2

In [21]:
df = pd.read_csv('data\\train.csv')
# test = pd.read_csv('test.csv')

train_cols = [i for i in df.columns if i not in ['def', 'id']]

train = df[train_cols]
y = df["def"] * 100

X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.25, stratify=y, shuffle=True, random_state=777)

In [22]:
xgb_clas = XGBClassifier()
xgb_reg = XGBRegressor()

xgb_clas.fit(X_train, y_train)
xgb_reg.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [28]:
roc_auc_score(y_test, xgb_clas.predict_proba(X_test)[:, 1])
# roc_auc_score(y_test, xgb_clas.predict(X_test))

0.6512380371208253

In [246]:
lgbm_params = {'booster': 'gbtree',
               'learning_rate': 0.06,
               'max_depth': 4,
#                'n_estimators': 130,
               'random_state': 777,
              }

logreg_params = {'penalty': 'l1',
                 'solver': 'liblinear', # 'lbfgs'
                 'C': 0.1,
                 'random_state': 777
                }

rnd_params = {'max_depth': 6,
              'n_estimators': 200,
              'random_state': 777
             }

knn_params ={'n_neighbors': 3,
             'algorithm': 'auto', # 'auto', 'kd_tree', 'brute'
             'p': 2,
            }

In [282]:
knn = KNeighborsClassifier(**knn_params)
rfc = RandomForestClassifier(**rnd_params)
lgbm = LGBMClassifier(**lgbm_params)
# log = LogisticRegression()

base_models = [knn, rfc, lgbm]
meta_model = LogisticRegression(**logreg_params)
type_base_models = 'classification' # типа базовых алгоритмов, либо 'regression'

stacking = EasyStacking(base_models, meta_model, type_base_models)

stacking.fit(X_train, y_train, p=0, cv=3, err=1, random_state=777)
# p - % деление выборки, для блендинга; 0 если нужен стекинг
# cv - кол-во фолдов для стекинга
# err - вес с которым добавляется нормальный шум к каждому метапризнаку

EasyStacking(meta_model=LogisticRegression(C=0.1, class_weight=None, dual=False,
                                           fit_intercept=True,
                                           intercept_scaling=1, l1_ratio=None,
                                           max_iter=100, multi_class='auto',
                                           n_jobs=None, penalty='l1',
                                           random_state=777, solver='liblinear',
                                           tol=0.0001, verbose=0,
                                           warm_start=False),
             models=[KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                          metric='minkowski',
                                          metric_pa...
                                    class_weight=None, colsample_bytree=1.0,
                                    importance_type='split', learning_rate=0.06,
                                    max_depth=4, min_child_samples=20,
 

In [283]:
roc_auc_score(y_test, stacking.predict_proba(X_test))

# base: lgbm, rnd | meta: logreg | rocauc = 0.67904
# base: lgbm, rnd | meta: logreg | rocauc = 0.67916


0.674993847624498

In [286]:
# Несколько блендингов подряд

blending = EasyStacking(base_models, meta_model, type_base_models)
predict_sum = 0

for t in range(1, 11):
    
    blending.fit(X_train, y_train, p=0.4, cv=0, err=0, random_state=None)
    
    predict_sum += blending.predict_proba(X_test)
    
    auc = roc_auc_score(y_test, predict_sum / t)
    print (auc)

0.6781864957597353
0.6784540648099402
0.6797029838625956
0.6811060416590914
0.6822867180884056
0.6828322638475877
0.6831177498799825
0.6824756368564971
0.6825697010129946
0.6825023805872659


In [263]:
# X, y = make_classification(100) # Построить на рандомной классификации